In [1]:
import os, sys

parentdir = os.path.abspath('/home/xcheng/mvesc/ETL')
sys.path.insert(0,parentdir)
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Image
from IPython.display import display
from numpy import random
import pickle
from estimate_prediction_model import *
from write_to_database import write_scores_to_db
from optparse import OptionParser
import re
%load_ext autotime
%matplotlib inline

In [18]:
tab_reports = 'model.reports' # it has batch_name, precision, recall
tab_feature_scores = 'model.feature_scores'
tab_prediction = 'model.predictions'
dir_pkls = '/mnt/data/mvesc/Models_Results/pkls'
pkls_logit = ['08_12_2016_grade_7_param_set_17_logit_ht_19082']
pkls_logit = os.path.join(dir_pkls, pkls_logit[0])+'_logit.pkl'
with open(pkls_logit, 'rb') as handle:
    pkl = pickle.load(handle)
    
with open(dir_pkls+'/08_12_2016_grade_7_param_set_17_logit_ht_19078_logit.pkl', 'rb') as handle:
    pkl2 = pickle.load(handle)
    

time: 49.9 ms


In [19]:
pkl

{'cross_validation_scores': array([ 0.24293077,  0.2596474 ]),
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'features': Index(['num_stem_classes_gr_5', 'num_future_prep_classes_gr_6',
        'num_health_classes_gr_5', 'percent_passed_pf_classes_gr_5', 'iss_gr_5',
        'avg_address_change_to_gr_5', 'n_districts_to_gr_5',
        'n_records_to_gr_5', 'avg_district_change_to_gr_5',
        'n_addresses_to_gr_5',
        ...
        'n_records_to_gr_6', 'num_humanities_classes_gr_6',
        'avg_city_change_to_gr_6', 'oss_gr_5', 'math_normalized_gr_5',
        'num_pf_classes_gr_5', 'num_art_classes_gr_5', 'gpa_gr_6',
        'stem_gpa_gr_5', 'num_health_classes_gr_6'],
       dtype='object', length=188),
 'file_name': '08_12_2016_grade_7_param_set_1

time: 58.8 ms


In [20]:
pkl.keys()

dict_keys(['test_y', 'model_name', 'val_set_preds', 'val_set_soft_preds', 'train_set_preds', 'train_y', 'train_set_soft_preds', 'test_set_preds', 'features', 'test_set_soft_preds', 'model_options', 'estimator', 'cross_validation_scores', 'time', 'parameter_grid', 'train_set_balance', 'file_name', 'val_y'])

time: 2.29 ms


In [3]:
# it seems the robusted-scaled are centered to median and rescaled by IQR;
# we may assume all features has a simiar scales

"""
Procedures to generate individual scores and top risk factors of logistic regression
1. load the features of current students at a certain grade;
2. load the corresonding model pickle file;
3. make predictions to get raw score (0, 1)
4. rank the scores and assiged risk level and risk score (5% high risk, 5% medium, 5% low; rescale 20% to 1 to 10);
5. find the top risk factors/features/column names;
6. save as pdf output to csv
"""
schema, table = 'model', 'individual_risk_scores_factors'
dir_pkls = '/mnt/data/mvesc/Models_Results/pkls'
pickle_file = '08_12_2016_grade_7_param_set_17_logit_ht_19082_logit.pkl'
if_exists = 'replace'
random_seed = 62571
topK = 3
both_positive_negative = True
num_students = 20
student_column = 'student_lookup'
pkl_model_key = 'estimator'

def topK_features_logit(model, data, feature_name, topN=3):
    importances = np.transpose(model.coef_)[:, 0]*data
    indices = importances.argsort()
    indices = indices[::-1]
    #print(indices[:3])
    return(list(np.array(features)[indices[:3]]))

with open(os.path.join(dir_pkls, pickle_file), 'rb') as handle:
    pkl = pickle.load(handle)
features = list(pkl['features']) # to pull feature data later

###!!! generate random all_features_dataframe with student_lookups for testing 
np.random.seed(random_seed)
all_data4prediction = np.random.rand(num_students, len(features))
all_data4prediction = pd.DataFrame(all_data4prediction, columns=features)
all_data4prediction[student_column] = range(1, all_data4prediction.shape[0]+1)
all_data4prediction = all_data4prediction[[student_column]+features]
###!!! random all_features_dataframe generated

student_lookups = all_data4prediction[student_column]
all_feature_data = all_data4prediction[features]
# all_feature_data = Robust_Scale(all_feature_data) #processed
risk_probas = pkl[pkl_model_key].predict_proba(all_feature_data)[:,1]
predictions = pkl[pkl_model_key].predict(all_feature_data)
top_individual_features = []
for i in range(all_feature_data.shape[0]):
    x = np.array(all_feature_data.iloc[i, :])
    top_individual_features.append(topN_features_logit(pkl[pkl_model_key], x, features, topN=3))

top_risk_factor_names = ['risk_factor_'+str(i) for i in range(1, topK+1)]
top_individual_features = pd.DataFrame(top_individual_features, columns=top_risk_factor_names)

# individual risk score & factors
individual_scores_factors = pd.DataFrame()
individual_scores_factors[student_column] = student_lookups
individual_scores_factors['risk_score'] =  risk_probas
individual_scores_factors = pd.concat([individual_scores_factors, top_individual_features], axis=1)

# individual risk facotrs values
top_feature_values = {'risk_factor_'+str(i):[] for i in range(1, topK+1)}
for risk_i in top_feature_values:
    for student_i in range(all_data4prediction.shape[0]):
        column_in_alldata = individual_scores_factors.ix[student_i, risk_i]
        top_feature_values[risk_i].append(all_data4prediction.ix[student_i, column_in_alldata])
top_feature_values = pd.DataFrame(top_feature_values)
top_feature_values = top_feature_values.rename(columns={x:x+'_value' for x in top_feature_values.columns})
individual_scores_factors = pd.concat([individual_scores_factors, top_feature_values], axis=1)

# model and its file name
individual_scores_factors['model'] = str(pkl[pkl_model_key])
individual_scores_factors['model_file'] = pickle_file

eng = postgres_engine_generator()
individual_scores_factors.to_sql(table, eng, schema = schema, if_exists=if_exists, index=False)

time: 3.37 s


In [4]:
individual_scores_factors.head()

,student_lookup,risk_score,risk_factor_1,risk_factor_2,risk_factor_3,risk_factor_1_value,risk_factor_2_value,risk_factor_3_value,model,model_file
0,1,0.758139,ethnicity_M,district_gr_6_Maysville,read_normalized_gr_5_isnull,0.722477,0.848921,0.876151,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
1,2,0.213100,ethnicity_M,disadvantagement_gr_6_economic,art_gpa_gr_6,0.416974,0.965883,0.760293,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
2,3,0.493108,ethnicity_M,art_gpa_gr_6,disadvantagement_gr_6_economic,0.654893,0.943058,0.789732,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
3,4,0.516004,ethnicity_M,district_gr_6_Maysville,read_normalized_gr_5_isnull,0.878060,0.996202,0.766150,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
4,5,0.792288,ethnicity_M,district_gr_6_Maysville,art_gpa_gr_6,0.900475,0.863978,0.803051,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...


time: 18.6 ms


In [88]:
def read_in_model(filename, model_name,
        pkl_dir = '/mnt/data/mvesc/Models_Results/pkls'):
    full_filename = filename +'_' + model_name + '.pkl'
    with open(os.path.join(pkl_dir, full_filename), 'rb') as model:
        model_pkl = pickle.load(model)
    clf, options = model_pkl['estimator'], model_pkl['model_options']
    return clf, options

def build_test_feature_set(options, current_year = 2016):
    # get student list of 2016 students in specified cohort grade level
    with postgres_pgconnection_generator() as connection:
        cohort = options['cohort_grade_level_begin']
        test_outcomes = read_table_to_df(connection,
            table_name = 'outcome', schema = 'model', nrows = -1,
            columns = ['student_lookup', 'current_students', cohort])
        test_outcomes.dropna(subset=['current_students', cohort], inplace=True)
        test_outcomes = pd.DataFrame(test_outcomes.student_lookup[
            test_outcomes[cohort] == current_year])

        for table, column_names in options['features_included'].items():
            features = read_table_to_df(connection, table_name = table,
                schema = 'model', nrows = -1,
                columns=(['student_lookup'] + column_names))
            # join to only keep features for current_students
            test_outcomes = pd.merge(test_outcomes, features,
                how = 'left', on = 'student_lookup')

    # build dataframe containing student_lookup
    # and all features as numeric non-categorical values
    test_outcomes.set_index('student_lookup', inplace=True)
    test_outcomes_raw = test_outcomes
    test_outcomes = df2num(test_outcomes_raw, drop_reference = False,
        dummify = True, drop_entirely_null = False)
    return test_outcomes_raw, test_outcomes

def test_impute_and_scale(test_outcomes, options):
    all_past_data = build_outcomes_plus_features(options)
    train, val, val = temporal_cohort_test_split(all_past_data,
            options['cohort_grade_level_begin'],
            options['cohorts_test'], options['cohorts_val'],
            options['cohorts_training'])
    exclude = set((options['outcome_name'],
                options['cohort_grade_level_begin']))
    train = train.drop([options['outcome_name'],
            options['cohort_grade_level_begin']],axis=1)
    val = val.drop([options['outcome_name'],
            options['cohort_grade_level_begin']],axis=1)

    category_missing = [col for col in train.columns if
                    col not in test_outcomes.columns]
    for col in category_missing:
        test_outcomes[col] = 0
    test_outcomes = test_outcomes.filter(train.columns)

    # imputation for missing values in features
    train, val, test_outcomes = impute_missing_values(train, val, test_outcomes,
        options['missing_impute_strategy'])

    # feature scaling
    train, val, test_outcomes = scale_features(train, val, test_outcomes,
        options['feature_scaling'])

    assert (all(train.columns == test_outcomes.columns)),\
        "train and current_students have different columns"
    return test_outcomes

def make_and_save_predictions(future_predictions, clf, filename):
    # generate soft predictions
    if hasattr(clf, "predict_proba"):
        future_set_scores = clf.predict_proba(future_predictions)[:,1]
    else:
        future_set_scores = clf.decision_function(future_predictions)

    saved_outputs = {
        'file_name' : filename,
        'future_index' : future_predictions.index,
        'future_scores' : future_set_scores,
        'future_preds' : clf.predict(future_predictions)
    }
    #write_scores_to_db(saved_outputs, importance_scores = False)
    return(saved_outputs)

def topK_features_logit(model, data, feature_names, topK=3):
    importances = np.transpose(model.coef_)[:, 0]*data
    indices = importances.argsort()
    indices = indices[::-1]
    #print(indices[:3])
    return(list(np.array(feature_names)[indices[:topK]]))

def risk_score2level(score, percentiles, risk_levels):
    ind = (percentiles>score).sum()
    return(risk_levels[ind])

parser = OptionParser()
parser.add_option('-f','--filename', dest='filename_list',
    help="filename for model to generate predictions",
    action="append")
(options, args) = parser.parse_args()

filename_list = ['08_12_2016_grade_7_param_set_17_logit_ht_19082']
topK = 3
schema, table = 'model', 'individual_risk_scores_factors'
dir_pkls = '/mnt/data/mvesc/Models_Results/pkls'
if_exists = 'replace'
random_seed = 62571

#if options.filename_list:
#    filename_list = options.filename_list

for filename in filename_list:
    # load saved model
    model_name = filename.split('_')[-3]
    clf, options = read_in_model(filename, model_name)
    
    # fetch and process feature data
    features_raw, feaures_num = build_test_feature_set(options)
    features_processed = test_impute_and_scale(feaures_num, options)
    
    # predict and find top factors
    risk_probas = clf.predict_proba(features_processed)[:,1]
    predictions = clf.predict(features_processed)
    top_individual_features = []
    for i in range(features_processed.shape[0]):
        x = np.array(features_processed.iloc[i, :])
        top_individual_features.append(topK_features_logit(clf, x, features_processed.columns, topK=topK))

    top_risk_factor_names = ['risk_factor_'+str(i) for i in range(1, topK+1)]
    top_individual_features = pd.DataFrame(top_individual_features, 
                                           columns=top_risk_factor_names)

    # individual risk score, level & factors
    individual_scores_factors = pd.DataFrame()
    individual_scores_factors['student_lookup'] = features_raw.index
    individual_scores_factors['risk_score'] =  risk_probas
    
    percentiles = individual_scores_factors.risk_score.quantile(q=[0.95, 0.85, 0.70])
    risk_levels = ['High', 'Medium', 'Low', 'Safe']
    student_risk_levels = [risk_score2level(s, percentiles, risk_levels) for s in individual_scores_factors.risk_score]
    individual_scores_factors['risk_level'] = student_risk_levels
    individual_scores_factors = pd.concat([individual_scores_factors, top_individual_features], axis=1)

    top_feature_values = {'risk_factor_'+str(i):[] for i in range(1, topK+1)}
    for risk_i in top_feature_values:
        for student_i in range(features_processed.shape[0]):
            column_in_features_processed = individual_scores_factors.ix[student_i, risk_i]
            top_feature_values[risk_i].append(features_processed[column_in_features_processed].iloc[student_i])
    top_feature_values = pd.DataFrame(top_feature_values)
    top_feature_values = top_feature_values.rename(columns={x:x+'_value' for x in top_feature_values.columns})
    individual_scores_factors = pd.concat([individual_scores_factors, top_feature_values], axis=1)

    # model and its file name
    individual_scores_factors['model'] = model_name
    individual_scores_factors['model_file'] = filename
    individual_scores_factors.sort_values(by=['risk_score'],inplace=True, ascending=False)
    eng = postgres_engine_generator()
    individual_scores_factors.to_sql(table, eng, schema = schema, if_exists=if_exists, index=False)

time: 26 s


In [89]:
individual_scores_factors

,student_lookup,risk_score,risk_level,risk_factor_1,risk_factor_2,risk_factor_3,risk_factor_1_value,risk_factor_2_value,risk_factor_3_value,model,model_file
1201,23990.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-8.005487,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1144,23779.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-8.005487,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
0,27587.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-4.386391,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
811,13237.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,read_normalized_gr_3,-17.094017,-6.002796,-2.571315,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1641,25517.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-14.153846,-4.704481,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1316,24139.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,read_normalized_gr_3,-18.461538,-6.308971,-1.191980,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
2158,28439.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-4.715497,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1683,25916.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-4.956846,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1272,24088.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-15.384615,-7.411328,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
2327,33950.0,1.000000,High,stem_gpa_gr_6,mid_year_withdraw_gr_5_nan,gpa_district_gr_6,-14.461538,1.000000,-2.608686,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082


time: 33.8 ms


In [90]:
individual_scores_factors

,student_lookup,risk_score,risk_level,risk_factor_1,risk_factor_2,risk_factor_3,risk_factor_1_value,risk_factor_2_value,risk_factor_3_value,model,model_file
1201,23990.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-8.005487,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1144,23779.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-8.005487,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
0,27587.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-4.386391,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
811,13237.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,read_normalized_gr_3,-17.094017,-6.002796,-2.571315,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1641,25517.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-14.153846,-4.704481,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1316,24139.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,read_normalized_gr_3,-18.461538,-6.308971,-1.191980,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
2158,28439.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-4.715497,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1683,25916.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-18.461538,-4.956846,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1272,24088.0,1.000000,High,stem_gpa_gr_6,gpa_district_gr_6,mid_year_withdraw_gr_5_nan,-15.384615,-7.411328,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
2327,33950.0,1.000000,High,stem_gpa_gr_6,mid_year_withdraw_gr_5_nan,gpa_district_gr_6,-14.461538,1.000000,-2.608686,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082


time: 34.1 ms


In [87]:
student_risk_levels

['High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 

time: 13.8 ms
